In [13]:
#Author: Mihai Avram, e-mail: mihai.v.avram@gmail.com

#for parsing logs
import json
#for connecting to the database
import psycopg2

#function for deciding on a score value to use for bot_score_english and bot_score_universal depending on what's available in the log
def score_decider(potential_score_keys, line_json):
    for key in potential_score_keys:
        if len(key) == 1:
            key1 = key[0]
            try:
                score = line_json[key1]
                print("proper key found")
                return score
            except:
                continue
        elif len(key) == 2:
            key1 = key[0]
            key2 = key[1]
            try:
                score = line_json[key1][key2]
                print("proper key found")
                return score
            except:
                continue
    
    return None



#connecting to the database
pgsqlconn = psycopg2.connect(host='localhost', user='postgres', password='password', dbname='botbase')
#cursor needed to execute db operations
botbase_cursor = pgsqlconn.cursor()

def num_requests_lookup(user_id):
    print(type(user_id))
    botbase_cursor.execute("""SELECT num_requests FROM public.botscore WHERE user_id=%s ORDER BY time_stamp DESC LIMIT 1;""", (user_id,))
    return botbase_cursor.fetchone()

def log_insertion_script(user_id, screen_name, time_stamp, all_bot_scores, bot_score_english, \
                bot_score_universal, requester_ip, tweets_per_day, num_tweets, \
                num_mentions, latest_tweet_timestamp, num_requests, user_profile):
    
    botbase_cursor.execute("""INSERT INTO public.botscore(
                user_id, screen_name, time_stamp, all_bot_scores, bot_score_english, 
                bot_score_universal, requester_ip, tweets_per_day, num_tweets, 
                num_mentions, latest_tweet_timestamp, num_requests, user_profile) 
                              VALUES 
                (%s, %s, to_timestamp(%s), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""", \
                (user_id, screen_name, time_stamp, json.dumps(all_bot_scores), bot_score_english, \
                bot_score_universal, requester_ip, tweets_per_day, num_tweets, \
                num_mentions, latest_tweet_timestamp, num_requests, user_profile))

    #commiting changes
    pgsqlconn.commit()

#parsing logs and uploading the entries to the botometer database
logFile = open("log-2.txt","r")

for line in logFile:
    line_json = json.loads(line)
    print("NEW ENTRY")
    user_id = line_json["search"]["user_id"]
    print("user_id:",line_json["search"]["user_id"])
    screen_name = line_json["search"]["sn"]
    print("screen_name:",line_json["search"]["sn"])
    time_stamp = line_json["timestamp"]
    print("time_stamp:",line_json["timestamp"])
    all_bot_scores = line_json["categories"]
    print("categories:",line_json["categories"])
    #english bot score which is either found in line_json["score"], line_json["classification"], line_json["score"]["english"]
    keys = [["score","english"],["score"],["classification"]]
    bot_score_english = score_decider(keys, line_json)
    print("bot_score_english:", bot_score_english)
    #universal bot score which is either found in line_json["score"]["universal"] or line_json["categories"]["languageagnostic_classification"] otherwise null
    keys = [["score","universal"],["categories","languageagnostic_classification"]]
    bot_score_universal = score_decider(keys, line_json)
    print("bot_score_universal:", bot_score_universal)
    requester_ip = line_json["remote_ip"]
    print("requester_ip:",str(line_json["remote_ip"]))
    tweets_per_day = None
    print("tweets_per_day:", "NULL")
    num_tweets = line_json["num_tweets"]
    print("num_tweets:", line_json["num_tweets"])
    num_mentions = None
    print("num_mentions:","NULL")
    latest_tweet_timestamp = None
    print("latest_tweet_timestamp:","NULL")
    num_requests = 1
    print("num_requests:", "lookup previous + 1")
    user_profile = None
    print("user_profile:","NULL")
    try:
        num_requests_new = num_requests_lookup(int(user_id))[0]
        num_requests_new = num_requests_new + 1
    except TypeError:
        num_requests_new = 1
    log_insertion_script(user_id, screen_name, time_stamp, all_bot_scores, bot_score_english, bot_score_universal, \
                str(requester_ip), tweets_per_day, num_tweets, num_mentions, latest_tweet_timestamp, num_requests_new, user_profile)
    
#closing access to database
botbase_cursor.close()
pgsqlconn.close()

#closing log file
logFile.close()



NEW ENTRY
user_id: 428845131
screen_name: awesomely_hk
time_stamp: 1435698355
categories: {'network_classification': 0.09, 'sentiment_classification': 0.46, 'friend_classification': 0.1, 'temporal_classification': 0.16, 'user_classification': 0.18, 'content_classification': 0.23}
proper key found
bot_score_english: 0.21
bot_score_universal: None
requester_ip: ['172.29.33.45', '137.132.3.8']
tweets_per_day: NULL
num_tweets: 200
num_mentions: NULL
latest_tweet_timestamp: NULL
num_requests: lookup previous + 1
user_profile: NULL
<class 'int'>
NEW ENTRY
user_id: 260101346
screen_name: mofuchannel
time_stamp: 1503234707
categories: {'network_classification': 0.09, 'sentiment_classification': 0.17, 'friend_classification': 0.21, 'languageagnostic_classification': 0.1, 'temporal_classification': 0.18, 'user_classification': 0.02, 'content_classification': 0.32}
proper key found
bot_score_english: 0.25
proper key found
bot_score_universal: 0.1
requester_ip: ['203.101.226.17']
tweets_per_day: N